In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

In [89]:
from datasets import load_dataset

eli5 = load_dataset("eli5", split="train_asks[:5000]")

Found cached dataset eli5 (/home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


Split the dataset's `train_asks` split into a train and test set with the [train_test_split](https://huggingface.co/docs/datasets/main/en/package_reference/main_classes#datasets.Dataset.train_test_split) method:

In [90]:
eli5 = eli5.train_test_split(test_size=0.2)

Loading cached split indices for dataset at /home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-1c80317fa3b1799d.arrow and /home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-bdd640fb06671ad1.arrow


In [91]:
eli5["train"][0]

{'q_id': '14p632',
 'title': 'Can someone explain why this is a valid method of measuring the obesity rate?',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['c7f5e1w'],
  'text': ["The article is using information directly from the CDC: [Prevalence of Obesity Among Children and Adolescents: United States, Trends 1963-1965 Through 2007-2008](_URL_1_). As far as I can tell, the cutoff for obesity was fixed to the 2000 charts, so it makes sense to compare obesity rates across different years. Those charts are based on data gathered from 1963-1994. (You can find the charts and methods [here](_URL_0_).) Notice that it hits around the exact 5% obesity (which you'd expect using the 95th percentile) in the '70s."],
  'score': [3]},
 'title_urls': {'url': []},
 'selftext_urls': {'url': []},
 'answers_urls': {'url': ['http://www.cdc.gov/growthcharts/',
   'http://www.cdc.gov/nchs/data/hestat/obesity_child_07_08/obesity_child_07_08.htm']}}

In [92]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/distilgpt2/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/f985248d2791fcff97732e4ee263617adec1edb5429a2b8421734c6d14e39bee.422318838d1ec4e061efb4ea29671cb2a044e244dc69229682bebd7cacc81631
Model config GPT2Config {
  "_name_or_path": "distilgpt2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "s

In [93]:
eli5 = eli5.flatten()
eli5["train"][0]

{'q_id': '14p632',
 'title': 'Can someone explain why this is a valid method of measuring the obesity rate?',
 'selftext': '',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['c7f5e1w'],
 'answers.text': ["The article is using information directly from the CDC: [Prevalence of Obesity Among Children and Adolescents: United States, Trends 1963-1965 Through 2007-2008](_URL_1_). As far as I can tell, the cutoff for obesity was fixed to the 2000 charts, so it makes sense to compare obesity rates across different years. Those charts are based on data gathered from 1963-1994. (You can find the charts and methods [here](_URL_0_).) Notice that it hits around the exact 5% obesity (which you'd expect using the 95th percentile) in the '70s."],
 'answers.score': [3],
 'title_urls.url': [],
 'selftext_urls.url': [],
 'answers_urls.url': ['http://www.cdc.gov/growthcharts/',
  'http://www.cdc.gov/nchs/data/hestat/obesity_child_07_08/obesity_child_07_08.htm']}

In [94]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

In [95]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-3eb13b9046685257_*_of_00004.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-0822e8f36c031199_*_of_00004.arrow


In [96]:
block_size = 128


def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [97]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-b38a088ca65ed389_*_of_00004.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa/cache-28df6ec4ce4a2bbd_*_of_00004.arrow


In [98]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [113]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("google/flan-t5-xxl")

loading configuration file https://huggingface.co/google/flan-t5-xxl/resolve/main/config.json from cache at /home/jovyan/.cache/huggingface/transformers/708951b885c9c9636b036da64baf3ab49511cc3fb3c36a6cba4b0bb5f215f4ed.ffdec67f8e5125f29c043e6647a780e92995b5d65e9460fe466aa976a31f1ab8
Model config T5Config {
  "_name_or_path": "google/flan-t5-xxl",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 10240,
  "d_kv": 64,
  "d_model": 4096,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 24,
  "num_heads": 64,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "torch_dtype": "float32",
  "transform

ValueError: Unrecognized configuration class <class 'transformers.models.t5.configuration_t5.T5Config'> for this kind of AutoModel: AutoModelForCausalLM.
Model type should be one of BartConfig, BertConfig, BertGenerationConfig, BigBirdConfig, BigBirdPegasusConfig, BlenderbotConfig, BlenderbotSmallConfig, BloomConfig, CamembertConfig, CodeGenConfig, CTRLConfig, Data2VecTextConfig, ElectraConfig, GPT2Config, GPTNeoConfig, GPTNeoXConfig, GPTJConfig, MarianConfig, MBartConfig, MegatronBertConfig, MvpConfig, OpenAIGPTConfig, OPTConfig, PegasusConfig, PLBartConfig, ProphetNetConfig, QDQBertConfig, ReformerConfig, RemBertConfig, RobertaConfig, RoFormerConfig, Speech2Text2Config, TransfoXLConfig, TrOCRConfig, XGLMConfig, XLMConfig, XLMProphetNetConfig, XLMRobertaConfig, XLMRobertaXLConfig, XLNetConfig.

In [100]:
training_args = TrainingArguments(
    output_dir="my_awesome_eli5_clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.773544
2,3.897600,3.755786
3,3.799800,3.751261


TrainOutput(global_step=1134, training_loss=3.8411788805872464, metrics={'train_runtime': 394.2435, 'train_samples_per_second': 68.927, 'train_steps_per_second': 2.876, 'total_flos': 887559737573376.0, 'train_loss': 3.8411788805872464, 'epoch': 3.0})

In [106]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 2216
  Batch size = 24


Perplexity: 42.57


In [107]:
trainer.save_model("trained_model")

Saving model checkpoint to trained_model
Configuration saved in trained_model/config.json
Model weights saved in trained_model/pytorch_model.bin


In [108]:
prompt = "Somatic hypermutation allows the immune system to"

In [109]:
from transformers import pipeline

generator = pipeline("text-generation", model="./trained_model", tokenizer=tokenizer)
generator(prompt)

loading configuration file ./trained_model/config.json
Model config GPT2Config {
  "_name_or_path": "./trained_model",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 6,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "torch_dtype"

[{'generated_text': "Somatic hypermutation allows the immune system to keep its own copy of the antigen.The immune system still thinks it has what the immune system needs when it tries to get rid of cells from one's own. In reality, a good copy of"}]

In [105]:
import torch
device = "cuda:0" 

In [83]:
model = model.to(device)

In [112]:
inputs = tokenizer("What allows somatic hypermutation?", return_tensors='pt')
inputs = inputs.to(device)
outputs = model.generate(**inputs)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['What allows somatic hypermutation?\n\nThe answer is yes.\n\nThe answer is']
